In [2]:
import pandas as pd, numpy as np
from API_calls_functions import *
from Analysis_functions import *
from tabulate import tabulate

In [ ]:
FedEX_API_Analysis_Useful_col=[['du_id','customer_id','street', 'street_2', 'postcode','city', 'state',
                                'street_R','street_2_R', 'postcode_R', 'city_R', 'state_R', 'customerMessages',
                                'SuiteRequiredButMissing', 'InvalidSuiteNumber', 'DPV']] # Did not include post-cleaning addresses
Other_API_input_col = [['du_id','customer_id','street', 'street_2', 'postcode','city', 'state']] 

## W4 Results Analysis

In [32]:
successful_all=pd.read_csv("data-W04/2024-W04 HF all PDL_post-FedExAPI-successful.csv")
# Add columns in DF
successful_all['fully_valid'] = np.where(
    (successful_all['DPV'] != True) |
    (successful_all['Resolved'] != True) |
    (successful_all['AddressType'] != "STANDARDIZED") |
    (successful_all['customerMessages'].apply(lambda x: len(str(x)) > 5)) |
    (successful_all['Interpolated'] == True),
    False,
    True)

successful_all['postcode_change'] = np.where(
    successful_all['postcode'].astype(int) != successful_all['postcode_R'].astype(int),
    True,
    False)

successful_all['city_change'] = np.where(
    successful_all['city_clean'].astype(str).str.upper() != successful_all['city_R'].str.upper(),
    True,
    False
)

successful_all['state_change'] = np.where(
    successful_all['state'].astype(str).str.upper() != successful_all['state_R'].str.upper(),
    True,
    False
)

successful_all=add_street_comparison(successful_all)

successful_all['Valid without change'] = (successful_all['fully_valid'] == True) & (successful_all['street_diff'] == False) & (successful_all['postcode_change'] == False) & (successful_all['state_change'] == False)
successful_all['Valid but need cleaning'] = (successful_all['fully_valid'] == True) & (successful_all['street_diff'] == True) & (successful_all['street_cleaning'] == True) & (successful_all['street_adding'] == False) & (successful_all['postcode_change'] == False) & (successful_all['state_change'] == False)
successful_all['Valid upon postcode change'] = (successful_all['postcode_change'] == True) & (successful_all['fully_valid'] == True) & (successful_all['state_change'] == False)
successful_all['Valid upon streetline completion'] = (successful_all['street_diff'] == True) & (successful_all['street_adding'] == True) & (successful_all['fully_valid'] == True) & (successful_all['postcode_change'] == False)
successful_all['Valid upon streetline correction'] = (successful_all['street_diff'] == True) & (successful_all['street_cleaning'] == False) & (successful_all['street_adding'] == False) & (successful_all['fully_valid'] == True) & (successful_all['postcode_change'] == False)
interpolated_error_message = str([{'code': 'INTERPOLATED.STREET.ADDRESS', 'message': 'Unable to confirm exact street number for the entered street name. The address falls within a valid range for the street name.'}])
successful_all['Invalid_Interpolated_Address'] = successful_all['customerMessages'].apply(lambda x: str(x)) == interpolated_error_message

#### Join with OTP Data

In [33]:
OTP_path = 'OTP_data_W48-W5.csv'
Spend_path = 'Address Correction Spend.csv'
OTP = pd.read_csv(OTP_path)
Nb_weeks = OTP['HELLOFRESH_WEEK'].nunique()
Spend = pd.read_csv(Spend_path)
print("Nb weeks",Nb_weeks)

Nb weeks 10


In [34]:
OTP.rename(columns={'DU_ID': 'du_id','BRAND': 'brand'}, inplace=True)
Spend=Spend.rename(columns={'Address Correction Approved Amount':'correction_charge','Tracking Number':'TRACKING_CODE'})
Spend['correction_charge'] = Spend['correction_charge'].replace('[\$,]', '', regex=True).astype(float)

OTP_charges=pd.merge(OTP, Spend[['correction_charge','TRACKING_CODE']], on='TRACKING_CODE',how='left').groupby(['du_id','BOX_ID','brand','HELLOFRESH_WEEK','OTP_GROUP','FINAL_ROOT_CAUSE']).agg({
    'correction_charge': 'sum'}).reset_index()

In [36]:
full_df=pd.merge(successful_all,OTP_charges,on='du_id', how='left')
full_df=full_df[full_df['OTP_GROUP'].notnull()] 
full_df['Address Flag'] = (full_df['FINAL_ROOT_CAUSE'].isin(["Address Issues", "Access Issues"])) | (full_df['correction_charge'] > 0)
len(full_df)

689445

### Percentage of L/U and Address RC flags

In [39]:
def get_LU(full_df):
    return full_df[full_df['OTP_GROUP'].isin(["Undelivered","Late_1","Late_2"])]

def get_Address_flags(full_df):
    return full_df[full_df['Address Flag']==True]

def get_charges(full_df):
    return full_df[full_df['correction_charge']>0]
    
def print_perc_LU_Addressflags(title, len_full_df,len_LU_df,len_LU_Address_df):
    print(f"{str(title)} ({len_full_df}) ||| Count of L/U: {len_LU_df}, Percentage: {round(len_LU_df/len_full_df*100, 2)}% ||| Count of L/U with Address issue: {len_LU_Address_df}, Percentage: {round(len_LU_Address_df/len_full_df*100, 2)}%")
    return

def print_table(list):
    table_data=[]
    for row in list:
        title, len_full_df, len_LU_df,len_Address_df, len_LU_Address_df,nb_charges,sum_charges=row
        table_data.append([title, len_full_df,
                           str("$")+str(round(sum_charges)),
                           nb_charges,
                           str(len_Address_df)+" ("+str(round(len_Address_df/len_full_df*100, 2))+"%)", 
                           str(len_LU_df)+" ("+str(round(len_LU_df/len_full_df*100, 2))+"%)", 
                           str(len_LU_Address_df)+" ("+str(round(len_LU_Address_df/len_full_df*100, 2))+"%)"
                           #str(sum_charges)+" ("+str(round(sum_charges/total_charges*100, 2))+"%)"
                           ])
    table_headers = ["Category", "Count", "Charges", "Charges Count", "Address Flags Count (%)", "L/U Count (%)",
                        "L/U+AddressFlags Count (%)"]

    print(tabulate(table_data, headers=table_headers, tablefmt="pretty"))
    return 

def get_row_stats(df,title):
    LU_df = get_LU(df)
    LU_Address_df = get_Address_flags(LU_df)
    charges_df = get_charges(df)
    
    row_stats_list = [[
            title,
            len(df),
            len(LU_df),
            len(get_Address_flags(df)),
            len(LU_Address_df),
            len(charges_df),
            sum(charges_df['correction_charge'])]]
    return row_stats_list

In [40]:
print(str("----"*14),"All boxes",str("----"*16))

all_boxes_stats=get_row_stats(full_df,"All boxes")
print_table(all_boxes_stats)

# -------------------

print(str("\n"+"----"*14),"FedEx Green Flags",str("----"*14))

Fully_valid_boxes=full_df[full_df['Valid without change']==True]
green_flag_stats=get_row_stats(Fully_valid_boxes,"Valid & Unchanged boxes")

Street_cleaning_boxes = full_df[full_df['Valid but need cleaning']==True]
green_flag_stats.extend(get_row_stats(Street_cleaning_boxes,"Valid & only street cleaning needed"))

print_table(green_flag_stats)

# -------------------


print(str("\n"+"----"*14),"FedEx Valid Upon Change Flag",str("----"*15))

Postcode_change_boxes = full_df[full_df['Valid upon postcode change']==True]
list_to_print=(get_row_stats(Postcode_change_boxes,"Postcode change boxes"))

Street_adding_boxes = full_df[full_df['Valid upon streetline completion']==True]
list_to_print.extend(get_row_stats(Street_adding_boxes,"Incomplete streetline boxes"))

Street_typo_boxes = full_df[full_df['Valid upon streetline correction']==True]
list_to_print.extend(get_row_stats(Street_typo_boxes,"Incorrect streetline boxes"))

print_table(list_to_print)

print(str("\n"+"----"*14),"FedEx Red Flags",str("----"*15))

Non_valid_boxes=full_df[full_df['DPV']!=True]
red_flag_stats=get_row_stats(Non_valid_boxes,"Non DPV Valid boxes")

Missing_apt_boxes = full_df[full_df['SuiteRequiredButMissing']==True] #NB: They are all also DPV Non Valid
red_flag_stats.extend(get_row_stats(Missing_apt_boxes,"Missing Apt boxes"))

Invalid_apt_boxes = full_df[full_df['InvalidSuiteNumber']==True]
red_flag_stats.extend(get_row_stats(Invalid_apt_boxes,"Invalid Apt boxes"))

Non_stand_boxes = full_df[full_df['AddressType'] != "STANDARDIZED"]
red_flag_stats.extend(get_row_stats(Non_stand_boxes,"Non standardized addresses boxes"))

Interpolated_boxes = full_df[full_df['Invalid_Interpolated_Address']==True]
red_flag_stats.extend(get_row_stats(Interpolated_boxes,"Interpolated addresses boxes"))
print_table(red_flag_stats)

-------------------------------------------------------- All boxes ----------------------------------------------------------------
+-----------+--------+---------+---------------+-------------------------+---------------+----------------------------+
| Category  | Count  | Charges | Charges Count | Address Flags Count (%) | L/U Count (%) | L/U+AddressFlags Count (%) |
+-----------+--------+---------+---------------+-------------------------+---------------+----------------------------+
| All boxes | 689445 | $13815  |      807      |      2445 (0.35%)       | 46610 (6.76%) |        1784 (0.26%)        |
+-----------+--------+---------+---------------+-------------------------+---------------+----------------------------+

-------------------------------------------------------- FedEx Green Flags --------------------------------------------------------
+-------------------------------------+--------+---------+---------------+-------------------------+---------------+-------------------

In [72]:
pd.set_option('display.max_columns', None)

display(Postcode_change_boxes[['street','street_2','postcode','city','state','street_R','street_2_R','postcode_R','city_R','state_R','DPV']])
pd.reset_option('display.max_columns')


,street,street_2,postcode,city,state,street_R,street_2_R,postcode_R,city_R,state_R,DPV
71,15444 Arroyo Dr,NaN,92697,Irvine,CA,15444 ARROYO DR,NaN,92617,IRVINE,CA,True
931,553 S Meadows Dr,NaN,85244,Chandler,AZ,553 S MEADOWS DR,NaN,85224,CHANDLER,AZ,True
1053,8887 TIERNEY CT,NaN,89148,Las vegas,NV,8887 TIERNEY CT,NaN,89149,LAS VEGAS,NV,True
1216,100 W Indian School Rd,Apt 4021,85012,Phoenix,AZ,100 W INDIAN SCHOOL RD,UNIT 4021,85013,PHOENIX,AZ,True
1367,33727 n wash view rd,NaN,85143,San tan valley,AZ,33727 N WASH VIEW RD,NaN,85144,SAN TAN VALLEY,AZ,True
...,...,...,...,...,...,...,...,...,...,...,...
687286,1206 4th place,NaN,32960,Vero Beach,FL,1206 4TH PL,NaN,32962,VERO BEACH,FL,True
691153,6565 Antelope Run Circle,Unit 302,80908,Colorado Springs,CO,6565 ANTELOPE RUN CIR,APT 302,80924,COLORADO SPRINGS,CO,True
691407,5380 Arroyo St,NaN,80920,Colorado Springs,CO,5380 ARROYO ST,NaN,80922,COLORADO SPRINGS,CO,True
691649,2559 e geddes pl,NaN,80123,centennial,CO,2559 E GEDDES PL,NaN,80122,LITTLETON,CO,True
